# 🔄 Crea tu propio RAG con LangChain, Chroma y Gradio
Este notebook construye una app RAG (Retrieval-Augmented Generation) que te permite subir un PDF o indicar una web, indexar el contenido y hacer preguntas en lenguaje natural.

In [9]:
!pip install langchain chromadb gradio pypdf unstructured tiktoken sentence-transformers

## Carga de documento PDF o desde web

In [13]:
import os
import openai
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from langchain.document_loaders import PyPDFLoader, WebBaseLoader

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")

# Cargar PDF
pdf_loader = PyPDFLoader("EU_AI_Act_ESP.pdf")  # Cambia por tu archivo
pdf_docs = pdf_loader.load()

# Cargar desde una web
# web_loader = WebBaseLoader("https://example.com")
# web_docs = web_loader.load()


OpenAI API Key exists and begins sk-proj-


## División en chunks y creación de base vectorial

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pdf_docs)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunks, embedding)


## Cadena de respuesta con contexto (RAG)

In [16]:
from langchain.llms import OpenAI  # Puedes cambiar por otro LLM gratuito como GPT4All o LlamaCpp
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),  # Usa tu clave API o sustituye por otro modelo
    retriever=db.as_retriever()
)

# Prueba
qa.run("¿Qué dice el documento sobre Inteligencia artificaial?")


' El documento habla sobre la evaluación de la documentación técnica relativa a un sistema de IA que un proveedor quiera introducir en el mercado o poner en servicio, y al que se aplicará un sistema de gestión de calidad mencionado en el punto 3. La solicitud debe incluir una declaración por escrito de que no se ha presentado la misma solicitud ante ningún otro organismo notificado y la documentación técnica prevista en el anexo IV.'

## Crear interfaz web con Gradio

In [21]:
import gradio as gr

def responder(pregunta, archivo):
    loader = PyPDFLoader(archivo.name)
    docs = loader.load()
    chunks = text_splitter.split_documents(docs)
    db = Chroma.from_documents(chunks, embedding)
    qa = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=db.as_retriever())
    return qa.run(pregunta)

demo = gr.Interface(
    fn=responder,
    inputs=[gr.Textbox(label="Pregunta"), gr.File(label="Sube tu PDF")],
    outputs="text",
    title="🔄 RAG OpenAI Chatbot"
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
